# Investigating the RIOT API

Using jupyter notebook to investigate the data returned from calls to the RIOT API

## Ensure the correct conda environment and kernel are being used
[following this guide](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/)


In [1]:
!conda env list 

# conda environments:
#
base                     /Users/chrisknowles/anaconda3
Riot_dev              *  /Users/chrisknowles/anaconda3/envs/Riot_dev
py311                    /Users/chrisknowles/anaconda3/envs/py311



In [2]:
!type python

python is /Users/chrisknowles/anaconda3/envs/Riot_dev/bin/python


 ## Accessing the API
 Create an account at [developer.riotgames.com](https://developer.riotgames.com/) to generate a development API key
 Keys last 24 hours

Get summoner info from SUMMONER-v4 api using summoner name
url = https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/ChrisK1987?api_key=RGAPI-c712ee61-5a8e-46ef-9c1d-cc5d5fbef63b

In [3]:
import requests
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Access API key
api_key = os.getenv('API_KEY')

#api_key = 'RGAPI-c712ee61-5a8e-46ef-9c1d-cc5d5fbef63b'
name = 'ChrisK1987'
api_SUMMONER_v4 = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'
summoner_info_url = f'{api_SUMMONER_v4}{name}?api_key={api_key}'
print(summoner_info_url)

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/ChrisK1987?api_key=RGAPI-d7263689-6143-45f4-ae12-eafaa9fe625e


In [4]:
#get request as .json
requests.get(summoner_info_url).json()

{'id': 'ZLvsYYSu3hAbPJ1nRN7ZlQ3L4Y2-QtPyhYpir3yV-gDEy03b',
 'accountId': 'PlzXQ0iYej21s0pN4ckX4pJ6uBXKvMdeAGLy0UREhy9C2pY',
 'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g',
 'name': 'ChrisK1987',
 'profileIconId': 3379,
 'revisionDate': 1699745156000,
 'summonerLevel': 434}

In [5]:
summoner_info = requests.get(summoner_info_url).json()

In [6]:
list(summoner_info.keys())

['id',
 'accountId',
 'puuid',
 'name',
 'profileIconId',
 'revisionDate',
 'summonerLevel']

In [7]:
#can probe the request for specific info using the dictionary key
summoner_info['puuid']

'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g'

## Using summoner_info to find match history and teamamtes



using MATCH-v5 api to get match IDs for my summoner_info puid and filtered on ranked games

In [8]:
api_MATCHES_v5 = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'

Its safter to use the api_key in the header then hide the api key in a .env file which can be .gitignore

In [9]:
summoner_matches_url = f"{api_MATCHES_v5}{summoner_info['puuid']}/ids?type=ranked&start=0&count=20"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": f"{api_key}"}


In [10]:
requests.get(summoner_matches_url, headers = headers)

<Response [200]>

In [11]:
#this is the 20 most recent ranked matches for the puuid passed in (me) including header for api key
requests.get(summoner_matches_url, headers = headers).json()

['EUW1_6648437306',
 'EUW1_6648386772',
 'EUW1_6648332079',
 'EUW1_6648244689',
 'EUW1_6646666103',
 'EUW1_6645391387',
 'EUW1_6645299672',
 'EUW1_6645164554',
 'EUW1_6644088095',
 'EUW1_6644029263',
 'EUW1_6643922033',
 'EUW1_6643863487',
 'EUW1_6635285868',
 'EUW1_6635214727',
 'EUW1_6635138424',
 'EUW1_6632385393',
 'EUW1_6632301060',
 'EUW1_6632214776',
 'EUW1_6632170555',
 'EUW1_6630873645']

In [12]:
summoner_matches = requests.get(summoner_matches_url, headers = headers).json()

Get the match info for a single match

In [29]:
api_MATCH_v5 = 'https://europe.api.riotgames.com/lol/match/v5/matches/'
match1_url = f'{api_MATCH_v5}{summoner_matches[0]}'
match1 = requests.get(match1_url, headers = headers).json()

In [ ]:
for i in len(summoner_matches):
    

In [36]:
extracted_info = []
for player in match1["info"]["participants"]:
        if player["puuid"] == summoner_info['puuid']:
            team_id = player["teamId"]
            break

for plater in match1["info"]["participants"]:
    extracted_info.append({
        "puuid": player["puuid"],
        "summonerName": player["summonerName"],
        "teamId": player["teamId"]
    })
print(extracted_info)

[{'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g', 'summonerName': 'ChrisK1987', 'teamId': 100}, {'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g', 'summonerName': 'ChrisK1987', 'teamId': 100}, {'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g', 'summonerName': 'ChrisK1987', 'teamId': 100}, {'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g', 'summonerName': 'ChrisK1987', 'teamId': 100}, {'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g', 'summonerName': 'ChrisK1987', 'teamId': 100}, {'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g', 'summonerName': 'ChrisK1987', 'teamId': 100}, {'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g', 'summonerName': 'ChrisK1987', 'teamId': 100}, {'puuid': 'z4qLmeJtju0fcmiPCTmo70

Create 
  